In [380]:
from pyfirmata import ArduinoMega as Arduino # import the right board but always call it Arduino
from pyfirmata import Pin
from pyfirmata import util
import numpy as np
import warnings
from tqdm import tqdm
import pandas as pd
import sys


In [418]:
def pin_id2n(pin_id): return str(mapping.query('pin_id  == "'+pin_id+'"')["pin_number"].iloc[0]).zfill(2)
pin_state2 = {p: "x" if v == 1 else " " for p, v in pin_state.items()}

    
def left_pair(p): return [pin_id2n(p), pin_state2[p]]
def show_circuit(pin_state, mapping, message=None, flush=True):
    pin_state = {p: "x" if v == 1 else " " for p, v in pin_state.items()}
    main_pins = mapping.query('pin_group == "main"')["pin_id"].values[1:]
    left_pins = mapping.query('pin_group == "left"')["pin_id"].values
    right_pins = mapping.query('pin_group == "right"')["pin_id"].values
    
    show_data = [pin_id2n(p) for p in main_pins] + [pin_state2[p] for p in main_pins]
    show_data2 = [left_pair(p) for p in left_pins] 
    show_data3 = [left_pair(p)[::-1] for p in right_pins] 
    show_data4 = list(zip(show_data2, show_data3))
    flatten = lambda l: [item for sublist in l for item in sublist]
    show_data4 = flatten(flatten(show_data4))
    show_data += show_data4
    n=4
    n2=18
    circuit = " " * 8 + "{}-VA  {}-MA  {}-IR  {}-VI   \n" +\
    " " * 8 + "[{}]" + " " * n + "[{}]" + " " * n + "[{}]" + " " * n + "[{}]\n" +\
    "{}-LO1  [{}]" + " " * n2 + "[{}] RO1-{}\n" +\
    "{}-LO2  [{}]" + " " * n2 + "[{}] RO2-{}\n" +\
    "{}-LES  [{}]" + " " * n2 + "[{}] RES-{}\n" +\
    "{}-LL   [{}]" + " " * n2 + "[{}]  RL-{}\n"
    filled_circuit = circuit.format(*show_data)        
        
        
    if not message is None:
        output = "{}: {}\n{}".format(datetime.datetime.now(), message, filled_circuit)
    else:
        output = filled_circuit
    if flush:
        nlines = len(filled_circuit.split('\n'))
              
        print(nlines)
        for i in range(nlines):
            sys.stdout.write("\033[F") #clear line
            sys.stdout.write("\033[K") #clear line
    sys.stdout.write(output)

In [461]:
program=pd.read_csv("program.csv")
program.head()

,pin_id,start,end,on,off
0,ONBOARD_LED,0,-1,1.0,4.0
1,MAIN_VALVE,0,-1,NaN,NaN
2,VACUUM,0,-1,NaN,NaN
3,IRLED,0,-1,NaN,NaN
4,VIBRATOR,0,2,0.1,0.1


In [462]:
mapping=pd.read_csv("pins_mapping.csv")
mapping

,pin_id,pin_number,pin_group
0,ONBOARD_LED,13,main
1,MAIN_VALVE,11,main
2,VACUUM,12,main
3,IRLED,2,main
4,VIBRATOR,50,main
5,LEFT_ODOUR_1,7,left
6,LEFT_ODOUR_2,9,left
7,LEFT_ESHOCK,5,left
8,LEFT_LED_R,3,left
9,RIGHT_ODOUR_1,8,right


In [463]:
pin_names = mapping.pin_id
pin_state = {p: 0 for p in pin_names}

In [467]:
program.query('start == 0 & end == -1 & on == on')


,pin_id,start,end,on,off
0,ONBOARD_LED,0,-1,1.0,4.0
9,LEFT_ESHOCK,0,-1,1.0,4.0
10,LEFT_LED_R,0,-1,0.1,0.9
15,RIGHT_ESHOCK,0,-1,1.0,4.0
16,RIGHT_LED_R,0,-1,0.1,0.9


In [477]:
def pin_daemon(pin_number, on, off, n_iters=None, start=None):
    
    if n_iters is None:
        n_iters = total_time // (on + off) # check if total time is accessible
        
    if not start is None:
        while np.abs((datetime.datetime.now() - start).total_seconds()) < 0.001:
            pass
    
    for _ in tqdm(range(n_iters)):
        start = datetime.datetime.now()

        board.digital[pin_number].write(1)
        pin_state[p] = 1
        runtime = datetime.datetime.now() - start
        if runtime.total_seconds() > 1.1*on:
            warnings.warn("Runtime: {}".format(runtime))
        board.digital[pin_number].write(0)
        pin_state[p]=0
        runtime = datetime.datetime.now() - start
        if runtime.total_seconds > 1.1*off:
            warnings.warn("Runtime: {}".format(runtime))
        # output a log to the root dir of the project! :)
        ## CODE
        ## Pretty cool a tqdm bar

            

        
def run(board, total_time=600):  
    program_start = datetime.datetime.now()
    #########################
    ## Constitutive pins
    #########################
    nan=np.nan
    const_pins = program.query('start == 0 & end == -1 & on != on')
    const_pins
    for p in program.pin_id:
        board.digital[p].write(1)
        pin_state[p] = 1

    #########################
    ## Constitutive - Intermitent pins
    #########################    
    interm_pins = program.query('start == 0 & end == -1 & on == on')
    # run a daemon for every pin
    constint_daemons = {}

    for p in interm_pins.pin_id:
        d = threading.Thread(name='daemon-{}'.format(p), target=pin_daemon, kwargs = {
            "pin_number" : mapping.query('pin_id == {}'.format(p))["pin_number"],
            "on" : program.query('pin_id == {}'.format(p))["on"],
            "off" : program.query('pin_id == {}'.format(p))["off"]
            })
        d.setDaemon(True)
        constint_daemons[p]=d

    
    #########################
    ## Scheduled - Intermitent pins
    #########################
    # They are not run throughout the lifetime of the program, just at some interval
    schint_pins = program.query('start == 0 & end != -1 & on == on')
    schint_daemons = {}
    
    count = {p: 0 for p in schint_pins.pin_id}
    for p in schint_pins.pin_id:
        d_on = program.query('pin_id == {}'.format(p))["on"]
        d_off = program.query('pin_id == {}'.format(p))["off"]
        d_start = program.query('pin_id == {}'.format(p))["start"]
        d_end = program.query('pin_id == {}'.format(p))["end"]
        d_name = 'daemon-{}_{}'.format(p, count[p])
        d = threading.Thread(name=d_name,
                             target=pin_daemon,
                             kwargs = {
            "pin_number": mapping.query('pin_id == {}'.format(p))["pin_number"],
            "on": d_on,
            "off": d_off,
            "n_iters": (d_end - d_start) / (d_on + d_off),
            "start": datetime.timedelta(seconds=d_start) + program_start
                             })
        d.setDaemon(True)
        schint_daemons[d_name]=d
        count[p] += 1

    
    #########################
    ## Scheduled pins
    #########################
    # They are not run throughout the lifetime of the program, just at some interval and without intermitency
    sch_pins = program.query('start == 0 & end != -1 & on != on')
    sch_daemons = {}
    
    count = {p: 0 for p in sch_pins.pin_id}
    for p in sch_pins.pin_id:
        d_on = program.query('pin_id == {}'.format(p))["on"]
        d_off = program.query('pin_id == {}'.format(p))["off"]
        d_start = program.query('pin_id == {}'.format(p))["start"]
        d_end = program.query('pin_id == {}'.format(p))["end"]
        d_name = 'daemon-{}_{}'.format(p, count[p])
        d = threading.Thread(name=d_name,
                             target=pin_daemon,
                             kwargs = {
            "pin_number": mapping.query('pin_id == {}'.format(p))["pin_number"],
            "on": d_on,
            "off": d_off,
            "n_iters": 1,
            "start": datetime.timedelta(seconds=d_start) + program_start,
                             })
        d.setDaemon(True)
        sch_daemons[d_name]=d
        count[p] += 1

    [d.start() for d in constint_daemons.values()]
    [d.start() for d in schint_daemons.values()]
    [d.start() for d in sch_daemons.values()]
    
    